<a href="https://colab.research.google.com/github/zhouxiaoqi0229/note/blob/main/movie%20recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pwd


/content


In [ ]:
import os
os.chdir('./drive/MyDrive/Colab Notebooks/movie_recommender')

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split#数据集划分
import numpy as np
from collections import Counter#用于统计字符出现的次数
import tensorflow as tf

import pickle #提供保存数据在本地的方法
import re #正则表达式
from tensorflow.python.ops import math_ops 
from urllib.request import urlretrieve#将url表示的网络对象复制到本地
from os.path import isfile,isdir#判断是否存在文件file 文件夹dir
from tqdm import tqdm  #Tqdm 是一个快速，可扩展的Python进度条，可以在 Python 长循环中添加一个进度提示信息，用户只需要封装任意的迭代器 tqdm(iterator)
import zipfile #用来做zip格式的压缩与解压缩
import hashlib #hash 或者MD5加密



In [ ]:
#数据查看

#用户数据
users_title = ['UserID', 'Gender', 'Age', 'OccupationID', 'Zip-code']
users = pd.read_csv('./users.dat',sep='::', header=None, names=users_title, engine = 'python')
users.info



<bound method DataFrame.info of       UserID Gender  Age  OccupationID Zip-code
0          1      F    1            10    48067
1          2      M   56            16    70072
2          3      M   25            15    55117
3          4      M   45             7    02460
4          5      M   25            20    55455
...      ...    ...  ...           ...      ...
6035    6036      F   25            15    32603
6036    6037      F   45             1    76006
6037    6038      F   56             1    14706
6038    6039      F   45             0    01060
6039    6040      M   25             6    11106

[6040 rows x 5 columns]>

In [ ]:
#电影数据
movies_title = ['MovieID', 'Title', 'Genres']
movies = pd.read_csv('./movies.dat', sep='::', header=None, names=movies_title, engine = 'python')
movies.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
#评分数据
ratings_title = ['UserID','MovieID', 'Rating', 'timestamps']
ratings = pd.read_csv('./ratings.dat', sep='::', header=None, names=ratings_title, engine = 'python')
ratings.head()

,UserID,MovieID,Rating,timestamps
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


**处理数据**


*   userID movieId occuption不变
*   性别 0,1 

*   年龄 0-6
*   类别;是分类字段，要转成数字。首先将Genres中的类别转成字符串到数字的字典，然后再将每个电影的Genres字段转成数字列表，因为有些电影是多个Genres的组合。


*   Title字段：处理方式跟Genres字段一样，首先创建文本到数字的字典，然后将Title中的描述转成数字的列表。另外Title中的年份也需要去掉
*   Genres和Title字段需要将长度统一，这样在神经网络中方便处理。空白部分用‘< PAD >’对应的数字填充。







In [ ]:
#数据预处理

def load_data():
  #修改user数据集
  users_title = ['UserID', 'Gender', 'Age', 'JobID', 'Zip-code']
  users = pd.read_csv('./users.dat',sep='::', header=None, names=users_title, engine = 'python')
  users = users.filter(regex='UserID|Gender|Age|JobID')
  users_orig = users.values
  users['Gender'] = users['Gender'].map({'F':0, 'M':1})#性别映射

  age_map = {val:ii for ii, val in enumerate(set(users['Age']))}
  users['Age'] = users['Age'].map(age_map)#年龄映射为0-6
  #修改movie数据集
  movies_title = ['MovieID', 'Title', 'Genres']
  movies = pd.read_csv('./movies.dat', sep='::', header=None, names=movies_title, engine = 'python')
  movies_orig = movies.values

  #去掉标题中的年份
  pattern = re.compile(r'^(.*)\((\d+)\)$')
  title_map = {val:pattern.match(val).group(1) for ii,val in enumerate(set(movies['Title']))}
  movies['Title'] = movies['Title'].map(title_map)

  #电影类型转换为数字字典
  genres_set = set()
  for val in movies['Genres'].str.split('|'):
    genres_set.update(val)
  genres_set.add('<PAD>')

  genres2int = {val:ii for ii, val in enumerate(genres_set)} #每种类型到一个数字的映射
  #eg: a|b|c:1|2|3这种类型的一个映射
  genres_map = {val:[genres2int[row] for row in val.split('|')] for ii,val in enumerate(set(movies['Genres']))} 
  #将类型补齐
  for key in genres_map:
    for cnt in range(max(genres2int.values())-len(genres_map[key])):
      genres_map[key].insert(len(genres_map[key])+cnt,genres2int['<PAD>'])
  movies['Genres'] = movies['Genres'].map(genres_map)
  movies.head(10)

  #电影Title转数字字典
  title_set = set()#所有电影名称涉及到的单词
  for val in movies['Title'].str.split():
    title_set.update(val)
  title_set.add('<PAD>')

  title_count = 15
  title2int = {val:ii for ii, val in enumerate(title_set)}#每个单词映射成一个数字
  title_map = {val:[title2int[row] for row in val.split()] for ii,val in enumerate(set(movies['Title']))}
  for key in title_map:
    for cnt in range(title_count - len(title_map[key])):
      title_map[key].insert(len(title_map[key]) + cnt,title2int['<PAD>'])
    
  movies['Title'] = movies['Title'].map(title_map)
  movies.head(5)

  #读取评分数据集
  ratings_title = ['UserID','MovieID', 'ratings', 'timestamps']
  ratings = pd.read_csv('./ratings.dat', sep='::', header=None, names=ratings_title, engine = 'python')
  ratings = ratings.filter(regex='UserID|MovieID|ratings')#不要时间戳

  #合并三个表
  data = pd.merge(pd.merge(ratings, users), movies)#1000209 rows × 8 columns
  #分成两张表
  target_fields = ['ratings']

  features_pd,targets_pd = data.drop(target_fields,axis=1),data[target_fields]
  features = features_pd.values #将dataframe转变为array
  targets_values = targets_pd.values

  return title_count, title_set, genres2int, features, targets_values, ratings, users, movies, data, movies_orig, users_orig

    



*   title_count:title字段的长度（15）

*   title_set：title文本的集合

*   genres2int：电影类型转数字的字典

*   features：是输入X
*   ratings：评分数据集的Pandas对象


*   movies_orig：没有做数据处理的原始电影数据


*   users_orig：没有做数据处理的原始用户数据




In [ ]:
title_count, title_set, genres2int, features, targets_values, ratings, users, movies, data, movies_orig, users_orig = load_data()

pickle.dump((title_count, title_set, genres2int, features, targets_values, ratings, users, movies, data, movies_orig, users_orig), open('preprocess.p', 'wb'))

In [ ]:
#辅助函数
def save_params(params):
  '''
  save params to filr
  '''
  pickle.dump(params,open('params.p','wb'))
def load_params():
    """
    Load parameters from file
    """
  return pickle.load(open('params.p', mode='rb'))


IndentationError: ignored

In [ ]:
import numpy as np
student = np.array([('name','S20'), ('age', 'i1'), ('marks', 'f4')]) 
student.take(1,0)

array(['age', 'i1'], dtype='<U5')

In [ ]:
#嵌入矩阵的维度
embed_dim = 32
#用户ID个数
uid_max = max(features.take(0,1)) + 1 # 6040+1
#性别个数
gender_max = max(features.take(2,1)) + 1 # 1 + 1 = 2
#年龄类别个数
age_max = max(features.take(3,1)) + 1 # 6 + 1 = 7
#职业个数
job_max = max(features.take(4,1)) + 1# 20 + 1 = 21

#电影ID个数
movie_id_max = max(features.take(1,1)) + 1 # 3952
#电影类型个数
movie_categories_max = max(genres2int.values()) + 1 # 18 + 1 = 19
#电影名单词个数
movie_title_max = len(title_set) # 5216
#电影名长度
sentences_size = title_count # = 15

#对电影类型嵌入向量做加和操作的标志
combiner = "sum"
#文本卷积滑动窗口，分别滑动2, 3, 4, 5个单词
window_sizes = {2, 3, 4, 5}
#文本卷积核数量
filter_num = 8

#电影ID转下标的字典，数据集中电影ID跟下标不一致，比如第5行的数据电影ID不一定是5
movieid2idx = {val[0]:i for i, val in enumerate(movies.values)}
movieid2idx

In [ ]:
#超参数设置
num_epochs= 5
bach_size =256
dropout_keep = 0.5
learning_rate = 0.0001
show_every_n_batches = 20

save_dir = './'

In [ ]:
# #定义输入的占位符
# def get_inputs():
  
uid = tf.keras.layers.Input(shape=(1,), dtype='int32', name='uid')  
user_gender = tf.keras.layers.Input(shape=(1,), dtype='int32', name='user_gender')  
user_age = tf.keras.layers.Input(shape=(1,), dtype='int32', name='user_age') 
user_job = tf.keras.layers.Input(shape=(1,), dtype='int32', name='user_job')

movie_id = tf.keras.layers.Input(shape=(1,), dtype='int32', name='movie_id') 
movie_categories = tf.keras.layers.Input(shape=(18,), dtype='int32', name='movie_categories') 
movie_titles = tf.keras.layers.Input(shape=(15,), dtype='int32', name='movie_titles') 
uid,user_gender,user_age,user_job,movie_id,movie_categories,movie_titles

(<KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'uid')>,
 <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'user_gender')>,
 <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'user_age')>,
 <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'user_job')>,
 <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'movie_id')>,
 <KerasTensor: shape=(None, 18) dtype=int32 (created by layer 'movie_categories')>,
 <KerasTensor: shape=(None, 15) dtype=int32 (created by layer 'movie_titles')>)

keras.layers.Embedding(input_dim,output_dim,input_length)


*   input_dim:词汇表的维度（有多少个不同词汇）
*   output_dim:嵌入词空间的维度（每个词嵌入为多少维度）


*   input_length:输入语句的长度（每一个词汇多少维）




In [ ]:
#test  embedding
from tensorflow import keras
import numpy as np

data = np.array([[0,0,0],[1,1,1]])
emb = keras.layers.Embedding(input_dim=2, output_dim=3, input_length=3) # 只有0,1所以input_dime是2
emb(data)


<tf.Tensor: shape=(2, 3, 3), dtype=float32, numpy=
array([[[ 1.0907508e-02,  9.1784373e-03, -2.6960148e-02],
        [ 1.0907508e-02,  9.1784373e-03, -2.6960148e-02],
        [ 1.0907508e-02,  9.1784373e-03, -2.6960148e-02]],

       [[ 2.1460298e-02,  3.7681412e-02,  4.0877610e-05],
        [ 2.1460298e-02,  3.7681412e-02,  4.0877610e-05],
        [ 2.1460298e-02,  3.7681412e-02,  4.0877610e-05]]], dtype=float32)>

In [ ]:
#定义用户的嵌入矩阵
#def get_user_embedding(uid,user_gender,user_age,user_job):
uid_embed_layer = tf.keras.layers.Embedding(uid_max, embed_dim,input_length=1,name='uid_embed_layer')
gender_embed_layer = tf.keras.layers.Embedding(gender_max, embed_dim//2,input_length=1,name='gender_embed_layer')
age_embed_layer = tf.keras.layers.Embedding(age_max,embed_dim//2,input_length=1, name='age_embed_layer')
job_embed_layer = tf.keras.layers.Embedding(job_max,embed_dim//2,input_length=1,name = 'job_embed_layer')

AttributeError: ignored


```
tf.keras.layers.Dense
(
    units,
    activation=None,
    use_bias=True,
    kernel_initializer='glorot_uniform',
    bias_initializer='zeros',
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    bias_constraint=None,
    **kwargs
)
```
tf.keras.layers.Dense相当于在全连接层中添加一个层

units:输出维度的大小



In [ ]:
def get_user_feature_layer(uid_embed_layer, gender_embed_layer, age_embed_layer, job_embed_layer):
    #第一层全连接
  uid_fc_layer = tf.keras.layers.Dense(embed_dim, name="uid_fc_layer", activation='relu')(uid_embed_layer)
  gender_fc_layer = tf.keras.layers.Dense(embed_dim, name="gender_fc_layer", activation='relu')(gender_embed_layer)
  age_fc_layer = tf.keras.layers.Dense(embed_dim, name="age_fc_layer", activation='relu')(age_embed_layer)
  job_fc_layer = tf.keras.layers.Dense(embed_dim, name="job_fc_layer", activation='relu')(job_embed_layer)

    #第二层全连接
  user_combine_layer = tf.keras.layers.concatenate([uid_fc_layer, gender_fc_layer, age_fc_layer, job_fc_layer], 2)  #(?, 1, 128)
  user_combine_layer = tf.keras.layers.Dense(200, activation='tanh')(user_combine_layer)  #(?, 1, 200)

  user_combine_layer_flat = tf.keras.layers.Reshape([200], name="user_combine_layer_flat")(user_combine_layer)##这一步不太明白
  return user_combine_layer, user_combine_layer_flat




TypeError: ignored

In [ ]:
#定义Movie Id的嵌入矩阵
def get_movie_id_embed_layer(movie_id):
  get_movie_id_embed_layer = tf.keras.layers.Embedding(movie_id_max,embed_dim,input_length=1,name='movie_id_embed_layer')
  return movie_id_embed_layer

In [ ]:
#合并电影类型的多个嵌入向量
def get_movie_categories_embed_layers(movie_categories):
  movie_categories_embed_layer = tf.keras.layers.Embedding(movie_categories_max,embed_dim,input_length=18,name='movie_categories_embed_layer')
  movie_categories_embed_layer = tf.kears.layers.Lambda(lambda layer:tf.reduce_sum(layer,axis=1,keepdims=True))#每一行求一个和
  return movie_categories_embed_layer


In [ ]:
#title 的文本卷积网络实现
movie_title_embed_layer = tf.kera